In [ ]:
import networkx as nx

In [ ]:
G = nx.karate_club_graph()

In [ ]:
pos = nx.spring_layout(G)
pos2 = {n: (pos[n][0], pos[n][1]) for n in pos}
node_indices = list(G.nodes)
startnodes = [s[0] for s in list(G.edges)]
endnodes = [s[1] for s in list(G.edges)]

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import GraphRenderer, Circle, StaticLayoutProvider, HoverTool, BoxZoomTool, ResetTool
from bokeh.palettes import Spectral8
from bokeh.plotting import figure,  output_notebook
output_notebook()

plot = figure(title='Graph Layout Demonstration', x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools='', toolbar_location=None)

graph = GraphRenderer()

graph.node_renderer.data_source.add(node_indices, 'index')
graph.node_renderer.glyph = Circle(size=5, fill_color='blue')

graph.edge_renderer.data_source.data = dict(start = startnodes, end = endnodes)

graph.layout_provider = StaticLayoutProvider(graph_layout=pos2)
node_hover_tool = HoverTool(tooltips=[("node_index", "$index")], renderers = [graph.node_renderer])
#edge_hover_tool = HoverTool(tooltips=[("start", "$start"), ("end", "$end")], renderers = [graph.edge_renderer])
plot.renderers.append(graph)
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())
output_file('graph.html')
show(plot)

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import GraphRenderer, Circle, StaticLayoutProvider
from bokeh.palettes import Spectral8
from bokeh.plotting import figure

plot = figure(title='Graph Layout Demonstration', x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools='', toolbar_location=None)

nodes = GraphRenderer()
#edges = GraphRenderer()

nodes.node_renderer.data_source.add(node_indices, 'index')
#graph.node_renderer.data_source.add(Spectral8, 'color')
nodes.node_renderer.glyph = Circle(size=15, fill_color='red')

nodes.edge_renderer.data_source.data = dict(
    start=startnodes,
    end=endnodes)

### start of layout code
#circ = [i*2*math.pi/8 for i in node_indices]
#x = [math.cos(i) for i in circ]
#y = [math.sin(i) for i in circ]

nodes.layout_provider = StaticLayoutProvider(graph_layout=pos2)

plot.renderers.append(nodes)
#plot.renderers.append(edges)

output_file('graph.html')
show(plot)


In [ ]:
import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models.graphs import from_networkx
output_notebook()

G = nx.karate_club_graph()

plot = figure(title="Karate Club Graph", tools="", x_range=(-1.5, 1.5),
          y_range=(-1.5, 1.5), toolbar_location=None)
graph = from_networkx(G, nx.spring_layout)
plot.renderers.append(graph)

x, y = zip(*graph.layout_provider.graph_layout.values())
node_labels = nx.get_node_attributes(G, 'club')
source = ColumnDataSource({'x': x, 'y': y,
                           'club': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='club', source=source,
                  background_fill_color='white')

plot.renderers.append(labels)
show(plot)

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import Button

output_file("button.html")

button = Button(label="Foo", button_type="success")

show(button)


In [ ]:
from datetime import date
from random import randint

from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, DataTable, DateFormatter, TableColumn

output_file("data_table.html")

data = dict(
        dates=[date(2014, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="dates", title="Date", formatter=DateFormatter()),
        TableColumn(field="downloads", title="Downloads"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

show(data_table)


In [22]:
from bokeh.models import ColumnDataSource, CustomJS, HoverTool
from bokeh.plotting import figure, output_file, show

output_file("hover_callback.html")

# define some points and a little graph between them
x = [2, 3, 5, 6, 8, 7]
y = [6, 4, 3, 8, 7, 5]
links = {
    0: [1, 2],
    1: [0, 3, 4],
    2: [0, 5],
    3: [1, 4],
    4: [1, 3],
    5: [2, 3, 4]
}

p = figure(plot_width=400, plot_height=400, tools="", toolbar_location=None, title='Hover over points')

source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1': []})
sr = p.segment(x0='x0', y0='y0', x1='x1', y1='y1', color='olive', alpha=0.6, line_width=3, source=source, )
cr = p.circle(x, y, color='olive', size=30, alpha=0.4, hover_color='olive', hover_alpha=1.0)

# Add a hover tool, that sets the link data for a hovered circle
code = """
const links = %s
const data = {'x0': [], 'y0': [], 'x1': [], 'y1': []}
const indices = cb_data.index.indices
for (var i = 0; i < indices.length; i++) {
    const start = indices[i]
    for (var j = 0; j < links[start].length; j++) {
        const end = links[start][j]
        data['x0'].push(circle.data.x[start])
        data['y0'].push(circle.data.y[start])
        data['x1'].push(circle.data.x[end])
        data['y1'].push(circle.data.y[end])
    }
}
segment.data = data
""" % links

callback = CustomJS(args={'circle': cr.data_source, 'segment': sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None, callback=callback, renderers=[cr]))

show(p)